In [1]:
import numpy as np, pandas as pd, re

In [2]:
class Preprocessing:
    def __init__(self, file_path):
        """
        Initialize the Preprocessing class with a file path and load it into a DataFrame.

        Parameters:
        file_path: str
            Path to the file to be processed.
        """
        
        self.file_path = file_path
        self.df = pd.read_csv(file_path)

    @staticmethod
    def return_null(val):
        """
        Replace values containing '__' or '_' in a string with nan.

        Parameters:
        val: The input value to be checked.

        Returns:
        nan if the condition is met, otherwise the original value.
        """
        
        if isinstance(val, str) and ("__" in val or "_" in val):
            return np.nan
        
        return val

    def process_invalid_numerical_columns(self):
        """
        Process specified numerical columns by replacing invalid values with nan and converting to float.

        Returns:
        pandas.DataFrame
            A new DataFrame with processed numerical columns based on the loaded CSV file.
        """
        
        new_df = self.df.copy()
        invalid_numerical_columns = [
            'Age',
            'Annual_Income',
            'Monthly_Inhand_Salary',
            'Num_of_Loan',
            'Num_of_Delayed_Payment',
            'Changed_Credit_Limit',
            'Outstanding_Debt',
            'Amount_invested_monthly',
            'Monthly_Balance'
        ]

        for col in invalid_numerical_columns:
            if col in new_df.columns:
                new_df[col] = new_df[col].apply(self.return_null)
                new_df[col] = new_df[col].astype('float')

        # Additional processing
        new_df["Age"] = new_df.Age.apply(lambda x: np.nan if x < 0 or x > 90 else x)
        new_df["Num_Bank_Accounts"] = new_df.Num_Bank_Accounts.apply(lambda x: np.nan if x < 0 or x > 15 else x)
        new_df["Num_Credit_Card"] = new_df.Num_Credit_Card.apply(lambda x: np.nan if x < 0 or x > 15 else x)
        new_df["Interest_Rate"] = new_df.Interest_Rate.apply(lambda x: np.nan if x > 50 else x)
        new_df["Num_of_Loan"] = new_df.Num_of_Loan.apply(lambda x: np.nan if x < 0 or x > 10 else x)
        new_df["Delay_from_due_date"] = new_df.Delay_from_due_date.apply(lambda x: 0.0 if x < 0 else x)
        new_df["Num_of_Delayed_Payment"] = new_df.Num_of_Delayed_Payment.apply(lambda x: np.nan if x < 0 or x > 30 else x)
        new_df["Changed_Credit_Limit"] = new_df.Changed_Credit_Limit.apply(lambda x: 0.0 if x < 0 else x)
        new_df["Num_Credit_Inquiries"] = new_df.Num_Credit_Inquiries.apply(lambda x: np.nan if x > 25 else x)
        new_df["Total_EMI_per_month"] = new_df.Total_EMI_per_month.apply(lambda x: np.nan if x > 1400 else x)

        return new_df

    @staticmethod
    def cal_history_age(val):
        """
        Calculate history age in months based on a string value containing years and months.
    
        Parameters:
        val: str
            The input value to be parsed.
    
        Returns:
        int or nan
            Total months calculated or nan if parsing fails.
        """
        
        try:
            year, month = 0, 0
            
            if re.search('year', val, re.IGNORECASE):
                year = int(re.findall(r'\d+', val)[0])
            
            if re.search('month', val, re.IGNORECASE):
                month = int(re.findall(r'\d+', val)[1])
            
            return year * 12 + month
        except:
            return np.nan

In [3]:
class Preprocessing:
    def __init__(self, file_path):
        # Hindari DtypeWarning
        self.df = pd.read_csv(file_path, low_memory=False)

    def process_invalid_numerical_columns(self):
        # Dummy: sesuaikan dengan kebutuhan sebenarnya
        return self.df.copy()

    @staticmethod
    def cal_history_age(x):
        try:
            years, months = 0, 0
            if isinstance(x, str):
                if 'Years' in x and 'Months' in x:
                    years = int(x.split('Years')[0].strip())
                    months = int(x.split('Years')[1].split('Months')[0].strip())
                elif 'Years' in x:
                    years = int(x.split('Years')[0].strip())
                elif 'Months' in x:
                    months = int(x.split('Months')[0].strip())
            return round(years * 12 + months, 2)
        except:
            return np.nan

# ==== START ====

file_path = "train.csv"
preprocessor = Preprocessing(file_path)

# Load and process
new_df = preprocessor.process_invalid_numerical_columns()
new_df['Credit_History_Age'] = new_df['Credit_History_Age'].apply(Preprocessing.cal_history_age)
new_df.drop(["ID", "Name", "SSN", "Annual_Income"], axis=1, inplace=True)

numerical_columns = [
    'Age',
    'Monthly_Inhand_Salary',
    'Num_Bank_Accounts',
    'Num_Credit_Card',
    'Interest_Rate',
    'Num_of_Loan',
    'Delay_from_due_date',
    'Num_of_Delayed_Payment',
    'Changed_Credit_Limit',
    'Num_Credit_Inquiries',
    'Outstanding_Debt',
    'Credit_Utilization_Ratio',
    'Total_EMI_per_month',
    'Amount_invested_monthly',
    'Monthly_Balance',
    'Credit_History_Age'
]

# Clean and fill numerical columns
def fill_numerical_missing_value(column, how, df):
    df = df.copy()

    # Optional: Remove symbols like $ or commas
    df[column] = df[column].astype(str).str.replace(r'[^\d\.\-]', '', regex=True)

    # Convert to float
    df[column] = pd.to_numeric(df[column], errors='coerce')

    missing_customer_ids = df[df[column].isna()]['Customer_ID'].unique()
    aggregated = df[df['Customer_ID'].isin(missing_customer_ids)].groupby('Customer_ID', as_index=False).agg({column: how})

    df = df.merge(aggregated, on='Customer_ID', how='left', suffixes=('', '_filled'))
    df.loc[df[column].isna(), column] = df.loc[df[column].isna(), f'{column}_filled']
    df.drop(columns=[f'{column}_filled'], inplace=True)

    return df[column]

# Apply for all numerical columns
for col in numerical_columns:
    new_df[col] = fill_numerical_missing_value(col, "median", new_df)

# Forward/backward fill by Customer_ID
new_df['Monthly_Balance'] = new_df.groupby('Customer_ID')['Monthly_Balance'].transform(lambda v: v.ffill().bfill())
new_df['Credit_History_Age'] = new_df.groupby('Customer_ID')['Credit_History_Age'].transform(lambda v: v.ffill().bfill())

# Final type conversion
new_df['Monthly_Balance'] = new_df['Monthly_Balance'].astype(float)
new_df['Credit_History_Age'] = new_df['Credit_History_Age'].astype(float)

# Handle categorical columns
categorical_columns = [
    'Month',
    'Occupation',
    'Type_of_Loan',
    'Credit_Mix',
    'Payment_of_Min_Amount',
    'Payment_Behaviour',
    'Credit_Score'
]

new_df['Occupation'] = new_df['Occupation'].apply(lambda x: np.nan if x == "_______" else x)
new_df['Credit_Mix'] = new_df['Credit_Mix'].str.replace('_', 'Unknown')
new_df['Payment_Behaviour'] = new_df['Payment_Behaviour'].str.replace('!@9#%8', 'Unknown')

# Clean loan types
new_df.loc[~new_df['Type_of_Loan'].isna(), 'Type_of_Loan'] = (
    new_df.loc[~new_df['Type_of_Loan'].isna(), 'Type_of_Loan']
    .apply(lambda x: str(x).split(',')[0])
)

# Fill missing Occupation and Loan Types
new_df['Occupation'] = new_df.groupby('Customer_ID')['Occupation'].transform(lambda v: v.ffill().bfill())
new_df['Type_of_Loan'] = new_df['Type_of_Loan'].fillna("Not Specified")

# Final cleaned DataFrame
new_df

,Customer_ID,Month,Age,Occupation,Monthly_Inhand_Salary,Num_Bank_Accounts,Num_Credit_Card,Interest_Rate,Num_of_Loan,Type_of_Loan,...,Credit_Mix,Outstanding_Debt,Credit_Utilization_Ratio,Credit_History_Age,Payment_of_Min_Amount,Total_EMI_per_month,Amount_invested_monthly,Payment_Behaviour,Monthly_Balance,Credit_Score
0,CUS_0xd40,January,23,Scientist,1824.843333,3,4,3,4,Auto Loan,...,Unknown,809.98,26.822620,0.0,No,49.574949,80.415295,High_spent_Small_value_payments,312.494089,Good
1,CUS_0xd40,February,23,Scientist,1824.843333,3,4,3,4,Auto Loan,...,Good,809.98,31.944960,0.0,No,49.574949,118.280222,Low_spent_Large_value_payments,284.629162,Good
2,CUS_0xd40,March,-500,Scientist,1824.843333,3,4,3,4,Auto Loan,...,Good,809.98,28.609352,0.0,No,49.574949,81.699521,Low_spent_Medium_value_payments,331.209863,Good
3,CUS_0xd40,April,23,Scientist,1824.843333,3,4,3,4,Auto Loan,...,Good,809.98,31.377862,0.0,No,49.574949,199.458074,Low_spent_Small_value_payments,223.451310,Good
4,CUS_0xd40,May,23,Scientist,1824.843333,3,4,3,4,Auto Loan,...,Good,809.98,24.797347,0.0,No,49.574949,41.420153,High_spent_Medium_value_payments,341.489231,Good
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,CUS_0x942c,April,25,Mechanic,3359.415833,4,6,7,2,Auto Loan,...,Unknown,502.38,34.663572,NaN,No,35.104023,60.971333,High_spent_Large_value_payments,479.866228,Poor
99996,CUS_0x942c,May,25,Mechanic,3359.415833,4,6,7,2,Auto Loan,...,Unknown,502.38,40.565631,NaN,No,35.104023,54.185950,High_spent_Medium_value_payments,496.651610,Poor
99997,CUS_0x942c,June,25,Mechanic,3359.415833,4,6,5729,2,Auto Loan,...,Good,502.38,41.255522,NaN,No,35.104023,24.028477,High_spent_Large_value_payments,516.809083,Poor
99998,CUS_0x942c,July,25,Mechanic,3359.415833,4,6,7,2,Auto Loan,...,Good,502.38,33.638208,NaN,No,35.104023,251.672582,Low_spent_Large_value_payments,319.164979,Standard


In [4]:
new_df.to_csv('train_cleaned.csv', index=False)